# The effect of pan-genome construction approach
This notebook contains the analysis of the effect of construction approach on pan-genome results.  
The analysis mainly consists of comparing three _A. thaliana_ pan-genomes, constructed with either the de novo (DN), the map-to-pan (MTP), or the iterative-assembly (IA) approach, based on the same 50x sequencing data and annotation evidence.

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from Bio import SeqIO
from itertools import chain

In [ ]:
pio.templates.default = "plotly_white"
colors = ['grey','purple','darkgreen','lightblue','orange']
pd.set_option("display.max_columns", None)

## Paths

In [ ]:
base_dir = "/groups/itay_mayrose_nosnap/liorglic/Projects/PGCM/output/A_thaliana_pan_genome"
dn_pg = os.path.join(base_dir, "de_novo/x50/RESULT")
mtp_pg = os.path.join(base_dir, "map_to_pan/x50/RESULT")
ia_pg = os.path.join(base_dir, "iterative_mapping/x50_bowtie2/RESULT")
compare_dir = os.path.join(base_dir, "compare_pan_genomes/DN_x50_vs_MTP_x50/RESULT")
compare_ia_mtp_dir = os.path.join(base_dir, "compare_pan_genomes/MTP_x50_vs_IA_x50_bowtie2/RESULT")

In [ ]:
dn_pav_tsv = os.path.join(dn_pg, "all_samples/pan_genome/pan_PAV.tsv")
mtp_pav_tsv = os.path.join(mtp_pg, "all_samples/pan_genome/pan_PAV.tsv")
ia_pav_tsv = os.path.join(ia_pg, "all_samples/pan_genome/pan_PAV.tsv")

In [ ]:
dn_nonref_analysis_dir = os.path.join(base_dir, 'analyze_nonref/DN_x50/RESULT')
mtp_nonref_analysis_dir = os.path.join(base_dir, 'analyze_nonref/MTP_x50/RESULT')

In [ ]:
figs_path = "/groups/itay_mayrose_nosnap/liorglic/Projects/PGCM/figs/FINAL"

## Basic stats comparison
Extract and compare general stats of the three pan-genomes

In [ ]:
# Read PAV TSVs as pandas data frames
dn_pg_pav_df = pd.read_csv(dn_pav_tsv, sep='\t', index_col=0)
dn_pg_pav_df.columns = [col.split('_')[0] for col in dn_pg_pav_df.columns]
mtp_pg_pav_df = pd.read_csv(mtp_pav_tsv, sep='\t', index_col=0)
mtp_pg_pav_df.columns = [col.split('_')[0] for col in mtp_pg_pav_df.columns]
ia_pg_pav_df = pd.read_csv(ia_pav_tsv, sep='\t', index_col=0)
ia_pg_pav_df.columns = [col.split('_')[0] for col in ia_pg_pav_df.columns]

In [ ]:
# Rename TAIR10 to col-0
dn_pg_pav_df.columns = [col if col != 'TAIR10' else 'Col-0' for col in dn_pg_pav_df.columns]
mtp_pg_pav_df.columns = [col if col != 'TAIR10' else 'Col-0' for col in mtp_pg_pav_df.columns]
ia_pg_pav_df.columns = [col if col != 'TAIR10' else 'Col-0' for col in ia_pg_pav_df.columns]

In [ ]:
# Calculate stats
def stats_from_pav_df(df):
    total_pangenes = df.shape[0]
    non_ref_pangenes = df.loc[df.index.str.startswith('PanGene')].shape[0]
    ref_pangenes = total_pangenes - non_ref_pangenes
    n_samples = df.shape[1]
    occup = df.sum(axis=1)
    core = (occup == n_samples).sum()
    shell = (occup.between(1,n_samples,inclusive='neither')).sum()
    singletons = (occup == 1).sum()
    index = ['Total pan-genes', 'Reference pan-genes', 'Non-reference pan-genes',
             'Core pan-genes', 'Shell pan-genes', 'Singletons']
    values =  [total_pangenes, ref_pangenes, non_ref_pangenes, core, shell, singletons]
    return pd.Series(values, index = index)

dn_pg_stats = stats_from_pav_df(dn_pg_pav_df)
mtp_pg_stats = stats_from_pav_df(mtp_pg_pav_df)
ia_pg_stats = stats_from_pav_df(ia_pg_pav_df)

In [ ]:
# Display stats
stats_df = pd.concat([dn_pg_stats, mtp_pg_stats, ia_pg_stats], axis=1)
stats_df.columns = ['De novo', 'Map-to-pan', 'Iterative assembly']
stats_df

In [ ]:
ref_nonref = stats_df.loc[['Reference pan-genes','Non-reference pan-genes']].transpose()
pg_composition = stats_df.loc[['Core pan-genes','Shell pan-genes','Singletons']].transpose()

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

fig_a_t1 = go.Bar(x=ref_nonref.index, y=ref_nonref['Reference pan-genes'], name='Reference', legendrank=5)
fig_a_t2 = go.Bar(x=ref_nonref.index, y=ref_nonref['Non-reference pan-genes'], name='Nonreference', legendrank=4)
fig.add_trace(fig_a_t1, row=1, col=1)
fig.add_trace(fig_a_t2, row=1, col=1)

fig_b_t1 = go.Bar(x=pg_composition.index, y=pg_composition['Core pan-genes'], name='Core', legendrank=3)
fig_b_t2 = go.Bar(x=pg_composition.index, y=pg_composition['Shell pan-genes'], name='Shell', legendrank=2)
fig_b_t3 = go.Bar(x=pg_composition.index, y=pg_composition['Singletons'], name='Singletons', legendrank=1)
fig.add_trace(fig_b_t1, row=1, col=2)
fig.add_trace(fig_b_t2, row=1, col=2)
fig.add_trace(fig_b_t3, row=1, col=2)

fig.update_layout(barmode='stack', colorway=colors, yaxis_title="Number of pan-genes")
fig.update_xaxes(mirror=True, showline=True, linecolor='black')
fig.update_yaxes(mirror=True, showline=True, linecolor='black', showgrid=False)
fig.show()

In [ ]:
fig2_a = os.path.join(figs_path, 'fig2a.pdf')
fig.write_image(fig2_a)

## Per sample reference/nonreference

In [ ]:
# Create table
dn_ref_nonref = dn_pg_pav_df.apply(lambda row: 'Reference' if row.name.startswith('transcript') else 'Nonreference', axis=1)
mtp_ref_nonref = mtp_pg_pav_df.apply(lambda row: 'Reference' if row.name.startswith('transcript') else 'Nonreference', axis=1)
ia_ref_nonref = ia_pg_pav_df.apply(lambda row: 'Reference' if row.name.startswith('transcript') else 'Nonreference', axis=1)

In [ ]:
def ref_nonref_per_sample(df, vec):
    per_sample = []
    for sample in df.columns:
        sample_pav = df[sample]
        sample_present = sample_pav.loc[sample_pav == 1]
        counts = pd.concat([sample_present, vec], axis=1, join='inner')[0].value_counts()
        counts.name = sample
        per_sample.append(counts)
    return pd.concat(per_sample, axis=1)

In [ ]:
dn_ref_nonref_per_sample = ref_nonref_per_sample(dn_pg_pav_df, dn_ref_nonref).fillna(0)
mtp_ref_nonref_per_sample = ref_nonref_per_sample(mtp_pg_pav_df, mtp_ref_nonref).fillna(0)
ia_ref_nonref_per_sample = ref_nonref_per_sample(ia_pg_pav_df, ia_ref_nonref).fillna(0)
# Order columns alphabetically
dn_ref_nonref_per_sample = dn_ref_nonref_per_sample[dn_ref_nonref_per_sample.columns.sort_values()]
mtp_ref_nonref_per_sample = mtp_ref_nonref_per_sample[mtp_ref_nonref_per_sample.columns.sort_values()]
ia_ref_nonref_per_sample = ia_ref_nonref_per_sample[ia_ref_nonref_per_sample.columns.sort_values()]
# Add _MTP / _IA suffices to columns names
mtp_ref_nonref_per_sample.columns = [col + '_1MTP' for col in mtp_ref_nonref_per_sample.columns]
ia_ref_nonref_per_sample.columns = [col + '_IA' for col in ia_ref_nonref_per_sample.columns]

ref_nonref_per_sample_df = pd.concat([dn_ref_nonref_per_sample,mtp_ref_nonref_per_sample,ia_ref_nonref_per_sample], axis=1)
ref_nonref_per_sample_df = ref_nonref_per_sample_df[ref_nonref_per_sample_df.columns.sort_values()]
ref_nonref_per_sample_df.columns = [col.split('_')[0] for col in ref_nonref_per_sample_df.columns]
ref_nonref_per_sample_df.columns = pd.MultiIndex.from_product([dn_ref_nonref_per_sample.columns,['De novo','Map-to-pan','Iterative assembly']])
# Make Col-0 the last column
ref_nonref_per_sample_df = ref_nonref_per_sample_df[pd.MultiIndex.from_tuples([x for x in ref_nonref_per_sample_df.columns if x[0] != 'Col-0'] + [x for x in ref_nonref_per_sample_df.columns if x[0] == 'Col-0'])]
ref_nonref_per_sample_df

In [ ]:
# Plot
dn_ref_nonref_per_sample_t = dn_ref_nonref_per_sample.transpose()
samples_order = [s for s in dn_ref_nonref_per_sample_t.index if s != 'Col-0'] + ['Col-0']
dn_ref_nonref_per_sample_t = dn_ref_nonref_per_sample_t.reindex(samples_order)

mtp_ref_nonref_per_sample_t = mtp_ref_nonref_per_sample.transpose()
mtp_ref_nonref_per_sample_t.index = [s.replace('_1MTP','') for s in mtp_ref_nonref_per_sample_t.index]
mtp_ref_nonref_per_sample_t = mtp_ref_nonref_per_sample_t.reindex(samples_order)

ia_ref_nonref_per_sample_t = ia_ref_nonref_per_sample.transpose()
ia_ref_nonref_per_sample_t.index = [s.replace('_IA','') for s in ia_ref_nonref_per_sample_t.index]
ia_ref_nonref_per_sample_t = ia_ref_nonref_per_sample_t.reindex(samples_order)

In [ ]:
fig = go.Figure()
x = [
    list(chain(*[[s]*3 for s in dn_ref_nonref_per_sample_t.index])),
    ['DN', 'MTP', 'IA']*len(dn_ref_nonref_per_sample_t.index)
]
y1 = list(chain(*zip(list(dn_ref_nonref_per_sample_t['Reference']),list(mtp_ref_nonref_per_sample_t['Reference']),list(ia_ref_nonref_per_sample_t['Reference']))))
y2 = list(chain(*zip(list(dn_ref_nonref_per_sample_t['Nonreference']),list(mtp_ref_nonref_per_sample_t['Nonreference']),list(ia_ref_nonref_per_sample_t['Nonreference']))))
fig.add_bar(name="Reference", x=x, y=y1, legendrank=2)
fig.add_bar(name="Nonreference", x=x, y=y2, legendrank=1)
fig.update_layout(barmode='stack', colorway=colors, yaxis_title="Number of pan-genes", bargap=0.1)
fig.update_xaxes(mirror=True, showline=True, linecolor='black')
fig.update_yaxes(mirror=True, showline=True, linecolor='black', showgrid=False)
fig.show()

In [ ]:
fig2_b = os.path.join(figs_path, 'fig2b.pdf')
fig.write_image(fig2_b)

## Occupancy analysis

In [ ]:
def occup_to_cat(occup, n_samples):
    if occup == 1:
        return "Singleton"
    elif occup < n_samples:
        return "Shell"
    elif occup == n_samples:
        return "Core"

def occup_categories_per_sample(df):
    occup = df.sum(axis=1)
    n_samples = df.shape[1]
    occup_cat = occup.apply(occup_to_cat, args=(n_samples,))
    
    per_sample = []
    for sample in df.columns:
        sample_pav = df[sample]
        sample_present = sample_pav.loc[sample_pav == 1]
        cat_counts = pd.concat([sample_present, occup_cat], axis=1, join='inner')[0].value_counts()
        cat_counts.name = sample
        per_sample.append(cat_counts)
    return pd.concat(per_sample, axis=1)

In [ ]:
dn_occup_cat_per_sample = occup_categories_per_sample(dn_pg_pav_df)
mtp_occup_cat_per_sample = occup_categories_per_sample(mtp_pg_pav_df)
ia_occup_cat_per_sample = occup_categories_per_sample(ia_pg_pav_df)
# Order columns alphabetically
dn_occup_cat_per_sample = dn_occup_cat_per_sample[dn_occup_cat_per_sample.columns.sort_values()]
mtp_occup_cat_per_sample = mtp_occup_cat_per_sample[mtp_occup_cat_per_sample.columns.sort_values()]
ia_occup_cat_per_sample = ia_occup_cat_per_sample[ia_occup_cat_per_sample.columns.sort_values()]
# Add _MTP suffices to columns names
mtp_occup_cat_per_sample.columns = [col + '_1MTP' for col in mtp_occup_cat_per_sample.columns]
ia_occup_cat_per_sample.columns = [col + '_IA' for col in ia_occup_cat_per_sample.columns]

occup_cat_per_sample_df = pd.concat([dn_occup_cat_per_sample,mtp_occup_cat_per_sample,ia_occup_cat_per_sample], axis=1)
occup_cat_per_sample_df = occup_cat_per_sample_df[occup_cat_per_sample_df.columns.sort_values()]
occup_cat_per_sample_df.columns = [col.split('_')[0] for col in occup_cat_per_sample_df.columns]
occup_cat_per_sample_df.columns = pd.MultiIndex.from_product([dn_occup_cat_per_sample.columns,['De novo','Map-to-pan','Iterative assembly']])
# Make Col-0 the last column
occup_cat_per_sample_df = occup_cat_per_sample_df[pd.MultiIndex.from_tuples([x for x in occup_cat_per_sample_df.columns if x[0] != 'Col-0'] + [x for x in occup_cat_per_sample_df.columns if x[0] == 'Col-0'])]
occup_cat_per_sample_df

## Nonreference gene pool
DN and MTP only

In [ ]:
nonref_matched = compare_dir + '/A_thaliana_DN_x50_vs_A_thaliana_MTP_x50_max_weight_matches.tsv'
nonref_matched_df = pd.read_csv(nonref_matched, sep='\t')

In [ ]:
# how many matched?
nonref_matched_df.shape

In [ ]:
nonref_matched_df.columns

### Nonreference genes reliability
Assessing the reliability of DN and MTP nonreference genes by homology searches and expression analysis.

In [ ]:
# Read tables
dn_nonref_tsv = os.path.join(dn_nonref_analysis_dir, 'nonref_analysis.tsv')
mtp_nonref_tsv = os.path.join(mtp_nonref_analysis_dir, 'nonref_analysis.tsv')
dn_nonref_df = pd.read_csv(dn_nonref_tsv, sep='\t', index_col=0)
mtp_nonref_df = pd.read_csv(mtp_nonref_tsv, sep='\t', index_col=0)

In [ ]:
# Basic stats

def nonref_stats(nonref_df, name='0'):
    stats = ['Total nonreference','High similarity to reference', 'Truncated reference',
         'Has plant homologs', 'Expressed', 'High quality candidates']
    tot_nonref = nonref_df.shape[0]
    high_sim_ref = nonref_df.query('identical_to_ref == 1').shape[0]
    trunc_ref = nonref_df.query('truncated_ref == 1').shape[0]
    has_homologs = nonref_df.query('has_homologs == 1').shape[0]
    expressed = nonref_df.query('expressed == 1').shape[0]
    hq = nonref_df.query('identical_to_ref == 0 and truncated_ref == 0 and (has_homologs == 1 or expressed == 1)').shape[0]
    stat_vals = [tot_nonref, high_sim_ref, trunc_ref, has_homologs, expressed, hq]
    return pd.DataFrame(stat_vals, index=stats, columns=[name])

In [ ]:
dn_nonref_stats = nonref_stats(dn_nonref_df, name='De novo')
mtp_nonref_stats = nonref_stats(mtp_nonref_df, name='Map-to-pan')
nonref_stats_df = pd.concat([dn_nonref_stats, mtp_nonref_stats], axis=1)
nonref_stats_df

In [ ]:
# Overlap with matched nonreference genes
dn_hq_candidates = dn_nonref_df.query(
    'identical_to_ref == 0 and truncated_ref == 0 and (has_homologs == 1 or expressed == 1)'
).index
dn_hq_candidates = set(dn_hq_candidates)
mtp_hq_candidates = mtp_nonref_df.query(
    'identical_to_ref == 0 and truncated_ref == 0 and (has_homologs == 1 or expressed == 1)'
).index
mtp_hq_candidates = set(mtp_hq_candidates)

n = nonref_matched_df.loc[nonref_matched_df['A_thaliana_DN_x50'].isin(dn_hq_candidates) & nonref_matched_df['A_thaliana_MTP_x50'].isin(mtp_hq_candidates)].shape[0]

print('Matched nonreference which are also HQ candidates in both DN and MTP:')
print(n)

### DN+|MTP- mapping
Analyzing the source of DN+|MTP- unmatched genes.
First, check how many DN+|MTP- transcripts can be mapped to MTP novel sequences (use minimap2 output). Then check how many transcripts can be mapped to the reference genome.

In [ ]:
# create fasta file of DN+|MTP- transcripts
dn_unmatched = set(dn_pg_pav_df.loc[(dn_pg_pav_df.index.str.startswith('PanGene')) & (~dn_pg_pav_df.index.isin(nonref_matched_df['A_thaliana_DN_x50']))].index)
dn_unmatched_recs = []
dn_trans_fasta = os.path.join(dn_pg, 'all_samples/pan_genome/pan_transcripts.fasta')
dn_unmatched_trans_fasta = os.path.join(compare_dir, 'MANUAL/DN_unmatched_transcripts.fasta')
for rec in SeqIO.parse(dn_trans_fasta, 'fasta'):
    if rec.id in dn_unmatched:
        dn_unmatched_recs.append(rec)

SeqIO.write(dn_unmatched_recs,dn_unmatched_trans_fasta,'fasta')

Mapping DN+|MTP- transcripts to MTP novel sequences, using Minimap2:

In [ ]:
# Load PAF
dn_unmatched_trans_vs_mtp_novel_paf = os.path.join(compare_dir, 'MANUAL/DN_unmatched_transcripts_vs_MTP_novel.paf')
paf_cols = ['Query_sequence_name', 'Query_sequence_length', 'Query_start',
            'Query_end', 'Relative_strand', 'Target_sequence_name',
            'Target_sequence_length', 'Target_start', 'Target_end',
            'Number_of_residue_matches', 'Alignment_block_length', 'Mapping_quality']
dn_unmatched_trans_vs_mtp_novel_df = pd.read_csv(dn_unmatched_trans_vs_mtp_novel_paf, sep='\t', header=None, usecols=range(12))
dn_unmatched_trans_vs_mtp_novel_df.columns = paf_cols
# Calculate query coverage
dn_unmatched_trans_vs_mtp_novel_df['Query_coverage'] = dn_unmatched_trans_vs_mtp_novel_df['Number_of_residue_matches'] / dn_unmatched_trans_vs_mtp_novel_df['Query_sequence_length']

In [ ]:
dn_unmatched_trans_mapped_to_mtp_novel = set(dn_unmatched_trans_vs_mtp_novel_df.query('Number_of_residue_matches/Query_sequence_length > 0.95')['Query_sequence_name'].unique())
n = len(dn_unmatched_trans_mapped_to_mtp_novel)
print('Number of DN+|MTP- genes successfully mapped to MTP novel:')
print(n)

Mapping DN+|MTP- transcripts to the reference genome, using Minimap2:

In [ ]:
dn_unmatched_trans_vs_ref_genome_paf = os.path.join(compare_dir, 'MANUAL/DN_unmatched_transcripts_vs_ref_genome.paf')
dn_unmatched_trans_vs_ref_genome_df = pd.read_csv(dn_unmatched_trans_vs_ref_genome_paf, sep='\t', header=None, usecols=range(12))
dn_unmatched_trans_vs_ref_genome_df.columns = paf_cols
# Calculate query coverage
dn_unmatched_trans_vs_ref_genome_df['Query_coverage'] = dn_unmatched_trans_vs_ref_genome_df['Number_of_residue_matches'] / dn_unmatched_trans_vs_ref_genome_df['Query_sequence_length']
# keep only mappings with highest query coverage per transcript
dn_unmatched_trans_vs_ref_genome_df = dn_unmatched_trans_vs_ref_genome_df.sort_values('Query_coverage', ascending=False).drop_duplicates('Query_sequence_name')

In [ ]:
# remove DN+|MTP- which were mapped to MTP novel sequences
dn_unmatched_trans_vs_ref_genome_df = dn_unmatched_trans_vs_ref_genome_df.loc[~ dn_unmatched_trans_vs_ref_genome_df['Query_sequence_name'].isin(dn_unmatched_trans_mapped_to_mtp_novel)]

In [ ]:
n = dn_unmatched_trans_vs_ref_genome_df.query('Query_coverage > 0.95').shape[0]
print('Number of DN+|MTP- genes successfully mapped to the reference genome:')
print(n)

Extract transcript mapping coordinates as BED and intersect with genes, to check if mapping to reference genes or to intergenic regions.

In [ ]:
bed_cols = ['Target_sequence_name', 'Target_start', 'Target_end', 'Query_sequence_name']
dn_unmatched_trans_map_to_ref_bed_df = dn_unmatched_trans_vs_ref_genome_df.query('Query_coverage > 0.95')[bed_cols]

In [ ]:
dn_unmatched_trans_map_to_ref_bed = os.path.join(compare_dir, 'MANUAL/DN_unmatched_transcripts_map_to_ref.bed')
dn_unmatched_trans_map_to_ref_bed_df.to_csv(dn_unmatched_trans_map_to_ref_bed, sep='\t', header=False, index=False)

Use bedtools intersect with reference genes. Require 50% overlap.

In [ ]:
dn_unmatched_trans_map_to_ref_genes_intersect_bed = os.path.join(compare_dir, 'MANUAL/DN_unmatched_transcripts_map_to_ref_gene_overlap_0.5.bed')
dn_unmatched_trans_map_to_ref_genes_intersect_bed_df = pd.read_csv(dn_unmatched_trans_map_to_ref_genes_intersect_bed, sep='\t', names=bed_cols)

In [ ]:
n = len(dn_unmatched_trans_map_to_ref_genes_intersect_bed_df['Query_sequence_name'].unique())
print('Number of DN+|MTP- genes successfully mapped to the reference genome and overlapping reference genes with at least 50%:')
print(n)

In [ ]:
dn_unmatched_trans_vs_ref_genome_df.query('Query_coverage > 0.2 & Query_coverage <= 0.95').shape

### DN-|MTP+ mapping
Analyze the source of DN-|MTP+ nonreference genes

In [ ]:
# create fasta files of DN-|MTP+ transcripts
mtp_unmatched = set(mtp_pg_pav_df.loc[(mtp_pg_pav_df.index.str.startswith('PanGene')) & (~mtp_pg_pav_df.index.isin(nonref_matched_df['A_thaliana_MTP_x50']))].index)
mtp_unmatched_recs = []
mtp_prot_fasta = os.path.join(mtp_pg, 'all_samples/pan_genome/pan_proteome.fasta')
mtp_unmatched_prot_fasta = os.path.join(compare_dir, 'MANUAL/MTP_unmatched_prot.fasta')
for rec in SeqIO.parse(mtp_prot_fasta, 'fasta'):
    if rec.id in mtp_unmatched:
        mtp_unmatched_recs.append(rec)

SeqIO.write(mtp_unmatched_recs,mtp_unmatched_prot_fasta,'fasta')

In [ ]:
# create fasta files of DN+|DN- and DN-|DN+ transcripts
dn_unmatched = set(dn_pg_pav_df.loc[(dn_pg_pav_df.index.str.startswith('PanGene')) & (~dn_pg_pav_df.index.isin(nonref_matched_df['A_thaliana_DN_x50']))].index)
dn_unmatched_recs = []
dn_prot_fasta = os.path.join(dn_pg, 'all_samples/pan_genome/pan_proteome.fasta')
dn_unmatched_prot_fasta = os.path.join(compare_dir, 'MANUAL/DN_unmatched_prot.fasta')
for rec in SeqIO.parse(dn_prot_fasta, 'fasta'):
    if rec.id in dn_unmatched:
        dn_unmatched_recs.append(rec)

SeqIO.write(dn_unmatched_recs,dn_unmatched_prot_fasta,'fasta')

Mapping DN-|MTP+ transcripts to DN+|MTP- transcripts, using Minimap2. This is done in order to look for truncated genes.

In [ ]:
# Read blast result
mtp_unmatched_prot_vs_dn_unmatched_prot_blast6 = "/groups/itay_mayrose_nosnap/liorglic/Projects/PGCM/output/A_thaliana_pan_genome/compare_pan_genomes/DN_x50_vs_MTP_x50/RESULT/MANUAL/MTP_unmatched_prot_vs_DN_unmatched_prot.blast6"
blast_cols = ["qseqid", "sseqid", "pident", "length", "mismatch",
              "gapopen", "qstart", "qend", "sstart", "send", "evalue",
              "bitscore", "qlen", "slen"]
mtp_unmatched_prot_vs_dn_unmatched_prot_df = pd.read_csv(mtp_unmatched_prot_vs_dn_unmatched_prot_blast6,
                                                        sep='\t', names=blast_cols)

In [ ]:
mtp_unmatched_prot_vs_dn_unmatched_prot_similar_df = mtp_unmatched_prot_vs_dn_unmatched_prot_df.query('pident > 95')
mtp_unmatched_prot_vs_dn_unmatched_prot_similar_df['qseqid'].unique().shape[0]

In [ ]:
len_ratios = mtp_unmatched_prot_vs_dn_unmatched_prot_similar_df['qlen']/mtp_unmatched_prot_vs_dn_unmatched_prot_similar_df['slen']
print(min(len_ratios), max(len_ratios))

Mapping DN-|MTP+ to proteins derived from annotations of 7 accessions

In [ ]:
# Read blast result
mtp_unmatched_prot_vs_dn_acc_prot_blast6 = "/groups/itay_mayrose_nosnap/liorglic/Projects/PGCM/output/A_thaliana_pan_genome/compare_pan_genomes/DN_x50_vs_MTP_x50/RESULT/MANUAL/MTP_unmatched_prot_vs_DN_all_acc_prot.blast6"
mtp_unmatched_prot_vs_dn_acc_prot_df = pd.read_csv(mtp_unmatched_prot_vs_dn_acc_prot_blast6,
                                                        sep='\t', names=blast_cols)

In [ ]:
mtp_unmatched_prot_vs_dn_acc_prot_df.query('pident > 95 & (qlen/slen > 0.9 | qlen/slen < 1.1)').loc[~ mtp_unmatched_prot_vs_dn_acc_prot_df['qseqid']
                                                                                      .isin(mtp_unmatched_prot_vs_dn_unmatched_prot_similar_df['qseqid'])]['qseqid'].unique().shape[0]

## Compare PAV matrices
To compare occupancies and detect PA discrepancies between the pan-genomes, we focus on reference and matched nonreference genes. Genes which are considered core in both pan-genomes were removed too.

In [ ]:
# remove unmatched genes
dn_pg_pav_matched_df = dn_pg_pav_df.loc[(~dn_pg_pav_df.index.str.startswith('PanGene')) | (dn_pg_pav_df.index.isin(nonref_matched_df['A_thaliana_DN_x50']))]
mtp_pg_pav_matched_df = mtp_pg_pav_df.loc[(~mtp_pg_pav_df.index.str.startswith('PanGene')) | (mtp_pg_pav_df.index.isin(nonref_matched_df['A_thaliana_MTP_x50']))]
# rename MTP matched nonreference to match DN
def tmp_func(x):
    if x in nonref_matched_df['A_thaliana_MTP_x50'].unique():
        return nonref_matched_df.loc[nonref_matched_df['A_thaliana_MTP_x50'] == x]['A_thaliana_DN_x50'].iloc[0]
    else:
        return x.replace(':','_')
mtp_pg_pav_matched_df.index = mtp_pg_pav_matched_df.index.map(tmp_func)
# sort rows and columns of PAV tables to get the same order
dn_pg_pav_matched_df.sort_index(inplace=True)
mtp_pg_pav_matched_df.sort_index(inplace=True)
dn_pg_pav_matched_df = dn_pg_pav_matched_df[dn_pg_pav_matched_df.columns.sort_values()]
mtp_pg_pav_matched_df = mtp_pg_pav_matched_df[mtp_pg_pav_matched_df.columns.sort_values()]

In [ ]:
assert all(dn_pg_pav_matched_df.columns == mtp_pg_pav_matched_df.columns) and all(dn_pg_pav_matched_df.index == mtp_pg_pav_matched_df.index)

In [ ]:
# Calculate occupancies
dn_pg_matched_occup = dn_pg_pav_matched_df.sum(axis=1)
mtp_pg_matched_occup = mtp_pg_pav_matched_df.sum(axis=1)
# Core sets
dn_pg_matched_core = set(dn_pg_matched_occup.loc[dn_pg_matched_occup == 8].index)
mtp_pg_matched_core = set(mtp_pg_matched_occup.loc[mtp_pg_matched_occup == 8].index)
# Core in both DN and MTP
both_matched_core = dn_pg_matched_core.intersection(mtp_pg_matched_core)
print("Number of genes which are core in both DN and MTP: %s" % len(both_matched_core))
print("Out of %s matched genes" % len(dn_pg_matched_occup))

In [ ]:
# remove genes which are core in both (keep noncore)
dn_pg_pav_matched_noncore_df = dn_pg_pav_matched_df.loc[~ dn_pg_pav_matched_df.index.isin(both_matched_core)]
mtp_pg_pav_matched_noncore_df = mtp_pg_pav_matched_df.loc[~ mtp_pg_pav_matched_df.index.isin(both_matched_core)]

In [ ]:
# Discrepancies table
discrep_df = dn_pg_pav_matched_noncore_df - mtp_pg_pav_matched_noncore_df
# Remove reference Col-0
discrep_df = discrep_df[[acc for acc in discrep_df.columns if acc != 'Col-0']]

In [ ]:
# discrep per sample
discrep_counts = []
for s in samples_order[:-1]:
    discrep_counts.append(discrep_df[s].value_counts()[[1,-1]])
discrep_counts_df = pd.concat(discrep_counts, axis=1).transpose()
discrep_counts_df.columns = ['DN+|MTP-','DN-/MTP+']
discrep_counts_df

In [ ]:
# Count discrepancies per gene
def count_discrep_types(row):
    val_counts = row.value_counts()
    for x in [0,-1,1]:
        if x not in val_counts:
            val_counts[x] = 0
    return val_counts.sort_index()

discrep_per_gene = discrep_df.apply(count_discrep_types, axis=1, result_type="expand")
discrep_per_gene.columns = ['DN-|MTP+', 'match', 'DN+|MTP-']

In [ ]:
discrep_per_gene.head()

In [ ]:
# How many with at least one discrepancy?
discrep_per_gene.query('match != 7').shape[0]

In [ ]:
# Sum across all genes
tot_discrep_types = discrep_per_gene.sum()
tot_pav_calls = tot_discrep_types.sum()
print("Total PAV calls: %s" % tot_pav_calls)
tot_discrep_types

#### MTP vs IA
PAV discrepancies between MTP and IA

In [ ]:
ia_mtp_nonref_matched = compare_ia_mtp_dir + '/A_thaliana_MTP_x50_vs_A_thaliana_IA_x50_bowtie2_max_weight_matches.tsv'
ia_mtp_nonref_matched_df = pd.read_csv(ia_mtp_nonref_matched, sep='\t')

In [ ]:
# remove unmatched genes
ia_pg_pav_matched_df = ia_pg_pav_df.loc[(~ia_pg_pav_df.index.str.startswith('PanGene')) | (ia_pg_pav_df.index.isin(ia_mtp_nonref_matched_df['A_thaliana_IA_x50_bowtie2']))]
mtp_pg_pav_matched_df = mtp_pg_pav_df.loc[(~mtp_pg_pav_df.index.str.startswith('PanGene')) | (mtp_pg_pav_df.index.isin(ia_mtp_nonref_matched_df['A_thaliana_MTP_x50']))]

In [ ]:
# replace ":" with "_" in ref gene names of IA
ia_pg_pav_matched_df.index = pd.Series(ia_pg_pav_matched_df.index).apply(lambda x: x.replace(':','_'))

In [ ]:
# rename MTP matched nonreference to match IA
def tmp_func(x):
    if x in ia_mtp_nonref_matched_df['A_thaliana_MTP_x50'].unique():
        return ia_mtp_nonref_matched_df.loc[ia_mtp_nonref_matched_df['A_thaliana_MTP_x50'] == x]['A_thaliana_IA_x50_bowtie2'].iloc[0]
    else:
        return x.replace(':','_')
mtp_pg_pav_matched_df.index = mtp_pg_pav_matched_df.index.map(tmp_func)

In [ ]:
# sort rows and columns of PAV tables to get the same order
ia_pg_pav_matched_df.sort_index(inplace=True)
mtp_pg_pav_matched_df.sort_index(inplace=True)
ia_pg_pav_matched_df = ia_pg_pav_matched_df[ia_pg_pav_matched_df.columns.sort_values()]
mtp_pg_pav_matched_df = mtp_pg_pav_matched_df[mtp_pg_pav_matched_df.columns.sort_values()]

In [ ]:
assert all(ia_pg_pav_matched_df.columns == mtp_pg_pav_matched_df.columns) and all(ia_pg_pav_matched_df.index == mtp_pg_pav_matched_df.index)

In [ ]:
# Calculate occupancies
ia_pg_matched_occup = ia_pg_pav_matched_df.sum(axis=1)
mtp_pg_matched_occup = mtp_pg_pav_matched_df.sum(axis=1)
# Core sets
ia_pg_matched_core = set(ia_pg_matched_occup.loc[ia_pg_matched_occup == 8].index)
mtp_pg_matched_core = set(mtp_pg_matched_occup.loc[mtp_pg_matched_occup == 8].index)
# Core in both DN and MTP
both_matched_core = ia_pg_matched_core.intersection(mtp_pg_matched_core)
print("Number of genes which are core in both IA and MTP: %s" % len(both_matched_core))
print("Out of %s matched genes" % len(ia_pg_matched_occup))

In [ ]:
# remove genes which are core in both (keep noncore)
ia_pg_pav_matched_noncore_df = ia_pg_pav_matched_df.loc[~ ia_pg_pav_matched_df.index.isin(both_matched_core)]
mtp_pg_pav_matched_noncore_df = mtp_pg_pav_matched_df.loc[~ mtp_pg_pav_matched_df.index.isin(both_matched_core)]

In [ ]:
# Discrepancies table
discrep_ia_mtp_df = ia_pg_pav_matched_noncore_df - mtp_pg_pav_matched_noncore_df
# Remove reference Col-0
discrep_ia_mtp_df = discrep_ia_mtp_df[[acc for acc in discrep_ia_mtp_df.columns if acc != 'Col-0']]

In [ ]:
# Count discrepancies per gene
discrep_per_gene_ia_mtp = discrep_ia_mtp_df.apply(count_discrep_types, axis=1, result_type="expand")
discrep_per_gene_ia_mtp.columns = ['IA-|MTP+', 'match', 'IA+|MTP-']

In [ ]:
discrep_per_gene_ia_mtp.shape

In [ ]:
# How many with at least one discrepancy?
discrep_per_gene_ia_mtp.query('match != 7').shape[0]

In [ ]:
# Sum across all genes
tot_discrep_types = discrep_per_gene_ia_mtp.sum()
tot_pav_calls = tot_discrep_types.sum()
print("Total PAV calls: %s" % tot_pav_calls)
tot_discrep_types

### Mapping DN-|MTP+ discrepancies
Mapping transcript sequences of DN-|MTP+ to the specific DN genomes in which they are missing.  
This allows us to determine whether they are missing because of assembly or because of annotation.

In [ ]:
# create required fastas with transcripts - fasta per accession
mtp_trans_fasta = os.path.join(mtp_pg, 'all_samples/pan_genome/pan_transcripts.fasta')
out_dir = compare_dir + '/MANUAL'
discrep_genes = {}
for acc in discrep_df.columns:
    discrep_genes[acc] = {}
    acc_discrep_genes = set(discrep_df.loc[discrep_df[acc] == -1].index)
    acc_discrep_genes = {g.replace("_",':') for g in acc_discrep_genes if not g.startswith('PanGene')}
    discrep_genes[acc]['genes'] = acc_discrep_genes
    discrep_genes[acc]['fh'] = open(os.path.join(out_dir,"%s_DN-_MTP+_discrep_trans.fasta" % acc), 'w')

for rec in SeqIO.parse(mtp_trans_fasta, 'fasta'):
    for acc in discrep_genes:
        if rec.id in discrep_genes[acc]['genes']:
            print(rec.format('fasta'), file=discrep_genes[acc]['fh'], end='')

for acc in discrep_genes:
    discrep_genes[acc]['fh'].close()

Mapping transcripts to 50x and HQ assemblies

In [ ]:
def mapped_transcripts(paf_path, min_map=0.9):
    paf_df = pd.read_csv(paf_path, sep='\t', header=None, usecols=range(12))
    paf_df.columns = paf_cols
    return set(paf_df.query('Number_of_residue_matches/Query_sequence_length >= @min_map')['Query_sequence_name'])

In [ ]:
mapped_trans = {}
for acc in discrep_df.columns:
    mapped_trans[acc] = {}

    paf_x50 = os.path.join(out_dir, '%s_DN-_MTP+_discrep_trans_vs_DN_genome.paf' % acc)
    paf_hq = os.path.join(out_dir, '%s_DN-_MTP+_discrep_trans_vs_HQ_genome.paf' % acc)
    
    mapped_trans[acc]['x50'] = mapped_transcripts(paf_x50)
    mapped_trans[acc]['HQ'] = mapped_transcripts(paf_hq)

In [ ]:
# How many DN-|MTP+ transcripts?
sum([len(discrep_genes[acc]['genes']) for acc in discrep_genes])

In [ ]:
# How many mapped to x50
tot_mapped_to_x50 = sum([len(mapped_trans[acc]['x50']) for acc in discrep_df.columns])
tot_mapped_to_x50

In [ ]:
# How many mapped to HQ assembly but not x50 assembly?
tot_mapped_to_hq = sum([len(mapped_trans[acc]['HQ'] - mapped_trans[acc]['x50']) for acc in discrep_df.columns])
tot_mapped_to_hq